In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

doc_df = pd.read_csv('tables/doc_69_76.csv')

In [3]:
messy_doc_topic_df = topic_model.get_document_info(free_text_list)

topic_desc_df = messy_doc_topic_df[['Name','Top_n_words']].drop_duplicates(ignore_index=True)
doc_topic_df = pd.DataFrame({'id_to_text':doc_df['id_to_text'],'assigned_topic':messy_doc_topic_df['Name']})

topic_desc_df.to_csv('tables/topic_descp_69_76.csv')
doc_topic_df.to_csv('tables/doc_topic_69_76.csv')

### construction below

In [ ]:
import gensim
import pyLDAvis.gensim_models as gensimvis

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

In [ ]:
txt ='Luis lives in Zurich.'

out_ = triplet_extractor(txt, return_tensors=True, return_text=False)[0]["generated_token_ids"]

In [ ]:
extracted_text = triplet_extractor.tokenizer.batch_decode(out_['output_ids'][0])

extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

In [ ]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def remove_persons(txt):
    document = nlp(txt)

    edited_txt = ""
    for ent in document:
        
        if ent.ent_type_=='PERSON':
            if ent.whitespace_:
                edited_txt += 'Person'+ ' '
            else:
                edited_txt += 'Person'
        else:
            if ent.whitespace_:
                edited_txt += ent.text+ ' '
            else:
                edited_txt += ent.text
    
    return edited_txt

In [ ]:
import pandas as pd
doc_df = pd.read_csv('tables/doc_69_76v30.csv')

In [ ]:
txt_list = doc_df['text'].values

In [ ]:
processed_txt_list = list(map(lambda x: remove_persons(x), txt_list))

In [ ]:
processed_txt_list[10]

In [ ]:
from keybert import KeyBERT

# Extract keywords
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(txt_list[0])



In [ ]:
import pickle

with open("plots/free_text_list", "rb") as fp:
        free_text_list = pickle.load(fp)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sentences = ["Relations between Greece and Turkey began in the 1830s following Greece's formation after its declaration of independence from the Ottoman Empire. Modern relations began when Turkey declared its formation in 1923 following the defeat of the Ottoman Empire in World War I. \
Greece and Turkey have a rivalry with a history of events that have been used to justify their nationalism.[1][2] These events include the population exchange between Greece and Turkey, the Istanbul pogrom and Cypriot intercommunal violence. Greek-Turkish feuding was not a significant factor in international relations from 1930 to 1955, and during the Cold War, domestic and bipolar politics limited competitive behaviour against each other.[3][4] By the mid-1990s and later decades, the restraint on their rivalry was removed, and both nations had become each other's biggest security risk.[5][6]\
Control of the eastern Mediterranean and Aegean seas remain the basis of the countries' rivalry. Following the end of World War II, the UNCLOS treaty, the decolonisation of Cyprus, and the addition of the Dodecanese to Greece's territory have caused turbulence in the relationship. Several issues frequently affect their current relations, including territorial disputes over the sea and air, minority rights, and Turkey's relationship with the European Union (EU) and its member states—especially Cyprus.[7][8] Control of energy pipelines is also an increasing focus in their relations."]

In [ ]:
for sentence in sentences:
    sid = SentimentIntensityAnalyzer()
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')

sentences = ["Relations between Greece and Turkey began in the 1830s following Greece's formation after its declaration of independence from the Ottoman Empire. Modern relations began when Turkey declared its formation in 1923 following the defeat of the Ottoman Empire in World War I. \
Greece and Turkey have a rivalry with a history of events that have been used to justify their nationalism.[1][2] These events include the population exchange between Greece and Turkey, the Istanbul pogrom and Cypriot intercommunal violence. Greek-Turkish feuding was not a significant factor in international relations from 1930 to 1955, and during the Cold War, domestic and bipolar politics limited competitive behaviour against each other.[3][4] By the mid-1990s and later decades, the restraint on their rivalry was removed, and both nations had become each other's biggest security risk.[5][6]\
Control of the eastern Mediterranean and Aegean seas remain the basis of the countries' rivalry. Following the end of World War II, the UNCLOS treaty, the decolonisation of Cyprus, and the addition of the Dodecanese to Greece's territory have caused turbulence in the relationship. Several issues frequently affect their current relations, including territorial disputes over the sea and air, minority rights, and Turkey's relationship with the European Union (EU) and its member states—especially Cyprus.[7][8] Control of energy pipelines is also an increasing focus in their relations."]

In [17]:
doc = nlp(sentences[0])

In [40]:
named_entities = []

entity_chunk = None #(entity,type)

for token in doc:
    if token.ent_iob_ == 'O':
        if entity_chunk:
            named_entities.append(entity_chunk)
            entity_chunk = None
    elif token.ent_iob_ == 'B':
        if entity_chunk:
            named_entities.append(entity_chunk)
            entity_chunk = None
        entity_chunk = (token.text,token.ent_type_)
    else:
        entity_chunk_text = entity_chunk[0]
        entity_chunk_type = entity_chunk[1]
        entity_chunk = (entity_chunk_text+' '+token.text,entity_chunk_type)


In [49]:
import numpy

np.unique(named_entities,axis=0)

array([['1923', 'DATE'],
       ['1930', 'DATE'],
       ['1955', 'DATE'],
       ['Aegean', 'NORP'],
       ['Cypriot', 'NORP'],
       ['Dodecanese', 'NORP'],
       ['EU', 'ORG'],
       ['Greece', 'GPE'],
       ['Greek - Turkish', 'NORP'],
       ['Istanbul', 'GPE'],
       ['Mediterranean', 'LOC'],
       ['Turkey', 'GPE'],
       ['UNCLOS', 'ORG'],
       ['World War I. Greece', 'EVENT'],
       ['World War II', 'EVENT'],
       ['the 1830s', 'DATE'],
       ['the Cold War', 'EVENT'],
       ['the European Union', 'ORG'],
       ['the Ottoman Empire', 'GPE'],
       ['the mid-1990s and later decades', 'DATE']], dtype='<U31')